# Osveževanje podatkov iz COBISS

Naloži funkcije za branje datotek XML iz COBISS.

In [3]:
%run 'cobiss.ipynb'

Read current list of researchers and publications.

In [4]:
%run 'data.ipynb'

Authors data in file: rd/_data/authors.yml
Number of records: 6678
Publications data in file: rd/_data/publications.yml
Number of records: 10332


In [5]:
import glob

V vsaki datoteki z update preveri, ali vsebuje kakšen nov podatek.

In [6]:
def read_one(fn):
    ret_authors = {}
    ret_publications = {}
    print(fn)
    for r in get_recs(fn):
        for a in r.get('avtorji', []):
            k = (a['surname'], a['firstname'], a['SICRIS'])
            ret_authors[k] = a
        ret_publications[r.get('COBISS')] = r
    print('{:} authors in {:}'.format(len(ret_authors), fn))
    print('{:} publications in {:}'.format(len(ret_publications), fn))
    return ret_authors, ret_publications

In [7]:
for fn in sorted(glob.glob('downloads/cobiss-org/*.xml.gz')):
    new_authors = {}
    new_publications = {}

    fbname = os.path.basename(fn).split('_cobiss')[0]
    cur_authors, cur_publications = read_one(fn)
    for a in sorted(cur_authors):
        if a not in authors:
            print("adding new author:", a)
            add_author(a)
            new_authors[a] = cur_authors[a]
    for p in cur_publications:
        if p not in publications:
            print(p)
            print("adding new publication:", cur_publications[p]['COBISS'], cur_publications[p]['naslov'])
            add_publication(cur_publications[p])
            new_publications[p] = cur_publications[p]
    print('total new added, authors: {:}, publications: {:}'.format(len(new_authors), len(new_publications)))
    print()

downloads/cobiss-org/2018-12-26-054807_cobiss.xml.gz
Processing group: ČLANKI IN DRUGI SESTAVNI DELI
Processing group: MONOGRAFIJE IN DRUGA ZAKLJUČENA DELA
Processing group: IZVEDENA DELA (DOGODKI)
Processing group: SEKUNDARNO AVTORSTVO
unexpected AuthorGroup: OrderedDict([('Corporate', 'Dnevi slovenske informatike (2009 ; Portorož)')])
record will be ignored: OrderedDict([('@bno', '6488'), ('@type', 'monograph'), ('AuthorGroup', OrderedDict([('Corporate', 'Dnevi slovenske informatike (2009 ; Portorož)')])), ('Title', 'Z inteligentnimi sistemi do strateške prednosti : zbornik \npovzetkov'), ('TitleISBD', 'Z inteligentnimi sistemi do strateške prednosti : zbornik \npovzetkov / Dnevi slovenske informatike 2009 - DSI, Portorož, Slovenija, \n15.-17. april 2009 ; [uredniški odbor Sašo Novaković ... et al.]'), ('Subtitle', 'zbornik povzetkov'), ('Publisher', OrderedDict([('Address', 'Ljubljana'), ('PublisherName', 'Slovensko društvo Informatika')])), ('PublisherISBD', 'Ljubljana : Slovensko 

Update latest publication date for each author.

In [8]:
for cobiss, p in publications.items():
    pub_date = p.get('PubDate', None)
    if pub_date is None:
        continue
    for a in p['avtorji']:
        k = (a['surname'], a['firstname'], a['SICRIS'])
        a_rec = authors[k]
        a_rec['latest_publication_year'] = max(a_rec.get('latest_publication_year', pub_date), pub_date)

Shrani končen seznam avtorjev in publikacij.

In [9]:
store_authors()
store_publications()

Uredi revije po kategorizaciji ARRS, impact faktorju in percentilu.

Hkrati preštej tudi avtorje.

In [12]:
avtorji = {}
ranking = []
for cobiss, p in publications.items():
    if 'JCR' not in p: continue
    rang = p['rang']
    leto, impact, per, impact_str, kats = get_JCR(p['JCR'])
#    if p['PubDate'] < 2019-5:
#        continue
    ranking.append((-p.get('CI', 0), rang, -impact, per, p['PubDate'], p['naslov']))
    
    if 'avtorji' not in p: continue
    for a in p['avtorji']:
        avtor = (a['surname'], a['firstname'], a['SICRIS'])
        avtorji[avtor] = avtorji.get(avtor, 0) + 1
ranking.sort()

Pet najpogostejših avtorjev:

In [14]:
sorted([(n, a) for a, n in avtorji.items()], reverse=True)[:20]

[(94, ('Zupan', 'Blaž', '12536')),
 (69, ('Bratko', 'Ivan', '02275')),
 (61, ('Leonardis', 'Aleš', '05896')),
 (59, ('Curk', 'Tomaž', '23399')),
 (58, ('Kononenko', 'Igor', '04242')),
 (43, ('Demšar', 'Janez', '16324')),
 (36, ('Jurič', 'Matjaž B.', '18337')),
 (34, ('Trček', 'Denis', '11077')),
 (29, ('Robnik Šikonja', 'Marko', '15295')),
 (29, ('Mraz', 'Miha', '13442')),
 (28, ('Kristan', 'Matej', '30155')),
 (28, ('Bajec', 'Marko', '16154')),
 (27, ('Zimic', 'Nikolaj', '05957')),
 (26, ('Jurišić', 'Aleksandar', '08724')),
 (25, ('Sadikov', 'Aleksander', '20389')),
 (24, ('Štrumbelj', 'Erik', '29486')),
 (24, ('Solina', 'Franc', '09581')),
 (24, ('Oblak', 'Polona', '22723')),
 (24, ('Kukar', 'Matjaž', '14565')),
 (23, ('Bosnić', 'Zoran', '28779'))]

Pet najboljših objav:

In [10]:
ranking[:20]

[(-88,
  "1A1 (Z, A'', A', A1/2)",
  -11.329,
  0.047619047619047616,
  2015,
  'The RNA-binding proteomes from yeast to man harbour conserved enigmRBPs'),
 (-70,
  "1A1 (Z, A'', A', A1/2)",
  -14.714,
  0.020689655172413793,
  2016,
  'Comprehensive identification of RNA-binding domains in human cells'),
 (-48,
  "1A1 (Z, A'', A', A1/2)",
  -9.413,
  0.08947368421052632,
  2016,
  'SR proteins are NXF1 adaptors that link alternative RNA processing to mRNA export'),
 (-45,
  "1A1 (Z, A', A1/2)",
  -2.033,
  0.48,
  2015,
  'Inertial sensor-based gait recognition : a review'),
 (-43,
  "1A1 (Z, A'', A', A1/2)",
  -19.381,
  0.026490066225165563,
  2015,
  'The RNA-binding protein HuR is essential for the B cell antibody response'),
 (-41,
  "1A1 (Z, A'', A', A1/2)",
  -14.018,
  0.020689655172413793,
  2014,
  'Rbfox2-coordinated alternative splicing of Mef2d and Rock2 controls myoblast fusion during myogenesis'),
 (-41,
  "1A1 (Z, A', A1/2)",
  -5.876,
  0.058823529411764705,
  2017,
 